<a href="https://colab.research.google.com/github/Akash743/Mathematical-Optimization/blob/main/Optimizer_weekly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyomo
!apt-get install -y -qq glpk-utils  #glpk is the solver
!wget -N -q "https://ampl.com/dl/open/couenne/couenne-linux64.zip"
!unzip -o -q couenne-linux64
import pyomo.environ as pyo
from pyomo.opt import SolverFactory

from pyomo.environ import * # for using pyomo functions like exp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd

model = pyo.ConcreteModel()

# Sets
model.i = pyo.RangeSet(1,6)
model.j = pyo.RangeSet(1,13)

# Parameters
S = pd.read_excel('Weekly_OPTIMIZATION_TESTING.xlsm',sheet_name='Spends',header=0,index_col=0,usecols='A:G',nrows=13)
C = pd.read_excel('Weekly_OPTIMIZATION_TESTING.xlsm',sheet_name='CPP',header=0,index_col=0,usecols='A:G',nrows=13)
T = pd.read_excel('Weekly_OPTIMIZATION_TESTING.xlsm',sheet_name='Transformation',header=0,index_col=0,usecols='A:G',nrows=13)

"""
i = M1, M2, M3...
j = W1, W2, W3...
"""

'\ni = M1, M2, M3...\nj = W1, W2, W3...\n'

In [27]:
C = C.fillna(0)
T = T.fillna(0)

In [28]:
T

,Type,Half Life,Max Adstock,Saturation,Steepness,Coefficient
1,Media,1.0,3.828436e+05,0.7,0.3,0.000835
2,Media,0.5,8.401198e+06,0.8,0.5,0.000069
3,Media,1.5,3.034951e+07,0.5,0.4,0.000006
4,Media,3.0,1.135283e+08,0.9,0.6,0.000022
5,Promo,0.0,7.274308e+03,0.0,0.0,0.045272
6,Media,3.8,7.525281e+02,0.6,1.5,0.251257


In [10]:
S.head()

,1,2,3,4,5,6
Period,,,,,,
1,1681.50,0.0,6141.681550,0.00,0.000000,0.000000
2,2476.10,0.0,15080.853226,12833.30,1547.846154,0.000000
3,3279.79,0.0,7192.357366,4191.13,1665.318681,39133.014118
4,2585.39,0.0,3574.155648,1806.90,1712.307692,39133.014118
5,1051.12,0.0,0.000000,800.00,1824.175824,5590.430588


In [3]:
spends = {}
for i in range(1,len(S.columns)+1):
  for j in range(1,len(S.index)+1):
    spends[(i,j)] = S.loc[j,i]
cpp = {} 
for i in range(1,len(C.columns)+1):
  for j in range(1,len(C.index)+1):
    cpp[(i,j)] = C.loc[j,i]

In [42]:
T.head()

,Type,Half Life,Max Adstock,Saturation,Steepness,Coefficient
1,Media,1.0,3.828436e+05,0.7,0.3,0.000835
2,Media,0.5,8.401198e+06,0.8,0.5,0.000069
3,Media,1.5,3.034951e+07,0.5,0.4,0.000006
4,Media,3.0,1.135283e+08,0.9,0.6,0.000022
5,Promo,NaN,7.274308e+03,NaN,NaN,0.045272


In [35]:
(1/2)^(1/int(T['Half Life']))

TypeError: ignored

In [31]:
T['Ret_rate'] = (1/2)^(1/T['Half Life'])

TypeError: ignored

In [29]:
hl = T['Half Life']
ma = T['Max Adstock']
satr = T['Saturation']
coef = T['Coefficient']
types = T['Type']
ret_rate = (1/2)^(1/hl)

TypeError: ignored

In [55]:
#model.del_component(model.cpk)

AttributeError: ignored

In [5]:
## Defining Parameters
##  ????? spends is param or var
# model.sp = pyo.Param(model.i,model.j, initialize = spends)
# sp = model.sp
model.cpm = pyo.Param(model.i,model.j, initialize = cpp)
cpm = model.cpm

model.hlf = pyo.Param(model.i,initialize = hl)
hlf = model.hlf

model.mad = pyo.Param(model.i,initialize = ma)
mad = model.mad

model.sat = pyo.Param(model.i,initialize = satr)
sat = model.sat

model.typ = pyo.Param(model.i,initialize = types)
typ = model.typ




default domain for Param objects is 'Any'.  However, we will be
changing that default to 'Reals' in the future.  If you really intend
the domain of this Paramto be 'Any', you can suppress this warning by
explicitly specifying 'within=Any' to the Param constructor.
(deprecated in 5.6.9, will be removed in (or after) 6.0)
(called from /usr/local/lib/python3.7/dist-packages/pyomo/core/base/indexed_component.py:645)
default domain for Param objects is 'Any'.  However, we will be
changing that default to 'Reals' in the future.  If you really intend
the domain of this Paramto be 'Any', you can suppress this warning by
explicitly specifying 'within=Any' to the Param constructor.
(deprecated in 5.6.9, will be removed in (or after) 6.0)
(called from /usr/local/lib/python3.7/dist-packages/pyomo/core/base/indexed_component.py:645)
default domain for Param objects is 'Any'.  However, we will be
changing that default to 'Reals' in the future.  If you really intend
the domain of this Paramto be 'Any

In [ ]:
model.rr = pyo.Param(model.i,initialize = types)
rr = model.rr

In [18]:
## Decision variable: spends
model.x = pyo.Var(model.i,model.j,domain = pyo.NonNegativeReals,initialize = spends)
x = model.x

In [23]:
print(x[1,1])

x[1,1]


In [9]:
for i in range(1,len(model.typ)):
  print(model.typ[i])

Media
Media
Media
Media
Promo


In [41]:
## Objective Function
def objective_rule(model):
  for i in range(1,len(model.typ)+1): ##i for var
    if model.typ[i] == "Media":
      


0.0

In [ ]:

from pyomo.environ import *

CPP = Spends/Raw Execution(RE)

RE = Spends/CPP

Adstocked execution = RE * Adstock factor(AF)
= (Spends/CPP)*AF

Avg adst execution = (Spends*AF/CPP)/Exec. periods

Now apply transformation on Avg adst execution to get TCP

Lift = TCP * Exec. periods * coeffcient

Put lift in Optimization equation



In [ ]:
## Defining the model
import numpy as np
#import math
from pyomo.environ import *
model = pyo.ConcreteModel()


sp1 = 33150.8411764706
sp2 = 233150.268146738


## Decision Variables
model.s1 = pyo.Var(within = pyo.NonNegativeReals,bounds=(.9*sp1,1.1*sp1)) 
s1 = model.s1
model.s2 = pyo.Var(within = pyo.NonNegativeReals,bounds=(.9*sp2,1.1*sp2))
s2 = model.s2


expre = ((1953.64352941177/(1+exp(-(10*3/1953.64352941177)*((s1*1/47/1)-0.179872867132035*1953.64352941177))))-8.81752993140885)*0.00133831972924945*47 + ((11441.0307432077/(1+exp(-(10*1/11441.0307432077)*((s2*1/1/154)-0.202998593210211*11441.0307432077))))-1328.19344200206)*0.00245568243410747*154

## Objective Function
model.Obj = pyo.Objective(expr =expre, sense = pyo.maximize)

## Constraint
model.Const1 = pyo.Constraint(expr = s1+s2<=266301.109323209)



optm = SolverFactory('couenne',executable='/content/couenne')
results = optm.solve(model)



print('objective function =', model.Obj())
print('s1 = ',model.s1())
print('s2 = ',model.s2())

print('sum total = ',model.s1()+model.s2())

objective function = 1065.7698788838547
s1 =  29835.757357158283
s2 =  236465.35226438538
sum total =  266301.10962154367


In [ ]:
# model.s3 = pyo.Var(within = pyo.NonNegativeReals,bounds=(.7*sp3,1.3*sp3))
# s3 = model.s3
# model.s4 = pyo.Var(within = pyo.NonNegativeReals,bounds=(.7*sp4,1.3*sp4))
# s4 = model.s4
# model.s5 = pyo.Var(within = pyo.NonNegativeReals,bounds=(.7*sp5,1.3*sp5))
# s5 = model.s5
# model.s6 = pyo.Var(within = pyo.NonNegativeReals,bounds=(.7*sp6,1.3*sp6))
# s6 = model.s6